1\)

Importing Data

In [4]:
import numpy as np

from sklearn import datasets
iris = datasets.load_iris()
X = iris.data[:,0:4] ## from this only take features 0,1,2,3
Y = iris.target

print(X.shape)
print(Y)

(150, 4)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [ ]:


def NB(X, Y):
    U = np.zeros((X.shape[1], 3))
    S = np.zeros((X.shape[1], 3))
    for i in range(3):
        for f in range(len(X.shape[1])):
            U[f,i] = X[np.where(Y==i),f].mean()
            S[f,i] = X[np.where(Y==i),f].std()
    

